In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import acquire as a
import prepare as p

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_mallcustomer_data():
    df = pd.read_sql('SELECT * FROM customers;', a.get_connection('mall_customers'))
    return df.set_index('customer_id')

In [3]:
df = get_mallcustomer_data()

In [4]:
df.head()

,gender,age,annual_income,spending_score
customer_id,,,,
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40


In [5]:
# from explore.py:
def train_validate_test_split(df, target, seed=1349):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed)
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed)
    return train, validate, test

In [6]:
def scale_my_data(train, validate, test):
    scaler = MinMaxScaler()
    scaler.fit(train[['age', 'annual_income']])
    X_train_scaled = scaler.transform(train[['age', 'annual_income']])
    X_validate_scaled = scaler.transform(validate[['age', 'annual_income']])
    X_test_scaled = scaler.transform(test[['age', 'annual_income']])

    train[['age_scaled', 'income_scaled']] = X_train_scaled
    validate[['age_scaled', 'income_scaled']] = X_validate_scaled
    test[['age_scaled', 'income_scaled']] = X_test_scaled
    return train, validate, test

In [ ]:
def prep_mall(df):
    '''
    dummy var for gender into is_male
    add 'spending_class' that cut spending score into the 4 quartiles and label the new field by [q1, q2, q3, q4]. 
    split on target of 'spending_score'
    scale age and annual income. 
    '''
    df['is_male'] = pd.get_dummies(df['gender'], drop_first=True)['Male']
    df = df.drop(columns=['gender'])
    df['spending_class'] = pd.qcut(df.spending_score, q=4, labels=['q1', 'q2', 'q3', 'q4'])
    train, validate, test = train_validate_test_split(df, target='spending_score', seed=1349)
    train, validate, test = scale_my_data(train, validate, test)
    return df, train, validate, test

In [7]:
df, train, validate, test = prep_mall(df)

**Goals of exploration**

- Can we see patterns, find signals in the data? 

- What features are driving the outcome?

- Are there other features we can construct that have stronger relationships? 

- Use Visualization and statistical testing to help answer these questions. 

- We want to walk away from exploration with with modeling strategies (feature selection, algorithm selection, evaluation methods, for example).  

**Agenda**

- Wrangle data (acquire, prep, summarize, split)

- Identify questions to answer, hypotheses to test.

In [ ]:
# planned, acquired, prepped 
#we are exploring

In [8]:
train.head()

,age,annual_income,spending_score,is_male,spending_class,age_scaled,income_scaled
customer_id,,,,,,,
27,45,28,32,0,q1,0.519231,0.106557
24,31,25,73,1,q3,0.250000,0.081967
40,20,37,75,0,q4,0.038462,0.180328
38,30,34,73,0,q3,0.230769,0.155738
57,51,44,50,0,q2,0.634615,0.237705


In [ ]:
#in the scope of exploration:
# - frame questions with our tools:
# univariate exploration
# bivariate exploration
#multivariate exploration

In [ ]:
# we will be creating bins for these features

In [ ]:
# 1. univariate explorate
# what are the distribution of each variables?

In [ ]:
p.distribution(df)

In [ ]:
#notes:
#spending score looks monstly notrmal
# income and age appear to be skewed right
#note skew on age an income if binning
#slight class imbalance on is male


In [ ]:
#exploration:
#
# Vizualization for exploration shoul be fast and infomrative.
#It serves the purpose of elaborating on relationshiop in you data and potential drivers of your target 

In [ ]:
#2.Bivariate:
#Does spending score vary across genders?

In [ ]:
#continous variable: spending score
#categorical variable : gender
# box plot to compare means visually
#do some statistical testingb

In [ ]:
sns.boxplot(train.is_male, train.spending_score)
plt.title(' Potential difference in spending score across gender?')
plt.show()

In [ ]:
# Null hypothesis: mean_ismale == mean_not_male
stats.levene(train[train.is_male==0].spending_score, train[train.is_male==1].spending_score)

In [ ]:
stats.ttest_ind(train[train.is_male==0].spending_score, train[train.is_male==1].spending_score, equal_var=True)

we cannot reject the null hypothesis and will continue to assertthat there is not a signifficant difference in the meansbetween the two groups

In [ ]:
# 3. Bivariate:
#is there a relationship between spending score and annual income?b

In [ ]:
print('Relationship between Income and Spending Score?')
sns.jointplot(x='annual_income', y='spending_score', data=train)
plt.xlabel('Income')
plt.ylabel('Spending Score')
plt.show()


In [ ]:
# note : looks like we might be able to create clusters from this variable (will explore more to fihure out how)
# no noted explicit linear correlation

In [ ]:
# 4. Is there a relationship between spending score and age

In [ ]:
plt.scatter(train.age, train.spending_score)
plt.xlabel('age')
plt.ylabel('Spending Score')
plt.title('Age vs Spending Score')
plt.show()

In [ ]:
#bin age
train['age_bin'] =pd.cut(train.age, [0,40,80])

In [ ]:
train.head()

In [ ]:
sns.boxplot(train.age_bin, train.spending_score)
plt.title('Spending Score across age bins')
plt.show()

In [ ]:
# levene H0 : equal variance across spending score between under 40 agroup and over 40 group
#alpha = 0.05
stats.levene(train[train.age <= 40].spending_score, train[train.age>40].spending_score)


In [ ]:
# reject the H0, varicna is not equal

In [ ]:
# H0: means_under_40 == mean_over_40
stats.ttest_ind(train[train.age <= 40].spending_score, train[train.age>40].spending_score, equal_var=False)

In [ ]:
#REJECT THE NULL HYPOTHESIS: WE HAVE AN IMPLIED DIFFERENCE BETWEEN THE AGE GROUPS OF THE SPENDING SCORE
#CONCLUSION: Difference between the means, no significant r value to be conclude

In [ ]:
#multivariate:
#5. If we control for age, does spending score differ across annual income

In [ ]:
sns.scatterplot(x='annual_income', y = 'spending_score', data = train[train.age<=40])
plt.title('Income vs Spending Score for those under up through 40 years old')
plt.show()

In [ ]:
sns.scatterplot(x='annual_income', y = 'spending_score', data = train[train.age>40])
plt.title('Income vs Spending Score for those over 40 years old')
plt.show()

In [ ]:
# possibly quadratic relationchip on income of those over 40 years old?

In [ ]:
sns.scatterplot(x='annual_income', y = 'spending_score', data = train[train.age<=30], color= 'blue')

sns.scatterplot(x='annual_income', y = 'spending_score', data = train[(train.age>30) & (train.age <= 40)], color ='red')
plt.title('Income vs Spending Score for those under 30 to those between 30 and 40  years old')
plt.show()

In [ ]:
# gender effecting this?


In [ ]:
sns.scatterplot(x='annual_income', y='spending_score', data = train[train.age<=40], hue ='is_male')
plt.title('Gender accounting for upper vs lower in age groups')
plt.show()

In [ ]:
#possible underlying differences that we have not observed. Potential quadratic relationship observed earlier in age brackets to note

In [ ]:
#6.if we control for income , does spending score differ across age?

In [ ]:
train['income_bin'] = pd.cut(train.annual_income, [0,40,70,140])

In [ ]:
train.head()

In [ ]:
train.income_bin.value_counts()

In [ ]:
sns.scatterplot(x='age', y='spending_score', data= train, hue = 'income_bin')
plt.title('Age to Spending Score controlling for Income Bins')
plt.show()

**takeaways**

- consistent spending score across age for middle class folks
- no strong observable correlation

In [ ]:
#if you want to do a parplot with significant amount data:
#utilizw.sample() to take randomized subset of 

In [ ]:
sns.pairplot(train, hue='age_bin')
plt.show()

In [ ]:
sns.pairplot(train, hue='is_male')
plt.show()

In [ ]:
sns.pairplot(train, hue='income_bin')
plt.show()

In [ ]:
#out of we have observed from our overall questions:
# No notable correlations observed from questions pointed thus far
#significant means difference across age brackets for spending score (drivers not yet known)
#Potential groupings on income brackets that coul lead to spending score

In [ ]:
#further exploration notes:
#see how we can leverage income brackets for modeling